In [1]:
import os.path as osp
from sys import argv
import cv2
import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
import subprocess
import random


def get_free_gpu():
    log = str(
        subprocess.check_output("nvidia-smi --format=csv --query-gpu=utilization.gpu,memory.used", shell=True)
    ).split(r"\n")[1:-1]
    free_gpu = []
    for idx, gpu_info in enumerate(log):
        if gpu_info[:-4].split(" %, ")[0] == "0" and gpu_info[:-4].split(" %, ")[1] == "3":
            free_gpu.append(idx)
    if free_gpu:
        return random.choice(free_gpu)
    raise RuntimeError("All gpus are used")


wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/27_seg/config.py")


set_random_seed(0, deterministic=False)
cfg.gpu_ids = [get_free_gpu()]
val = True
# Build dataset

# datasets = build_dataset(cfg.data.train)
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=val)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=13.08s)
creating index...
index created!
loading annotations into memory...
Done (t=3.90s)
creating index...
index created!


2023-05-22 03:32:43,348 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=5.53s)
creating index...


2023-05-22 03:32:49,508 - mmdet - INFO - Start running, host: aiarhipov@lorien.atp-fivt.org, work_dir: /home/aiarhipov/centernet/exps/27_seg
2023-05-22 03:32:49,510 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH 

index created!


2023-05-22 03:32:54,460 - mmdet - WARNING - No meta information found in the runner. 
wandb:   12 of 12 files downloaded.  
2023-05-22 03:33:49,650 - mmdet - INFO - Epoch [1][14/1372]	lr: 1.748e-05, eta: 2:27:07, time: 3.233, data_time: 0.225, memory: 2448, loss_center_heatmap: 46.8994, loss_offset: 83.8865, loss_c2v: 79.8060, loss_v2c: 39.9089, loss: 250.5008, grad_norm: 215.0152
2023-05-22 03:34:54,874 - mmdet - INFO - Epoch [1][28/1372]	lr: 3.497e-05, eta: 2:58:37, time: 4.659, data_time: 0.026, memory: 2448, loss_center_heatmap: 24.9838, loss_offset: 82.7336, loss_c2v: 78.7215, loss_v2c: 39.3654, loss: 225.8043, grad_norm: 136.6417
2023-05-22 03:35:46,763 - mmdet - INFO - Epoch [1][42/1372]	lr: 5.245e-05, eta: 2:54:06, time: 3.706, data_time: 0.024, memory: 2448, loss_center_heatmap: 9.5302, loss_offset: 83.4564, loss_c2v: 79.4201, loss_v2c: 39.7181, loss: 212.1247, grad_norm: 71.7836
2023-05-22 03:36:38,993 - mmdet - INFO - Epoch [1][56/1372]	lr: 6.993e-05, eta: 2:51:41, time: 3.7

In [4]:
datasets[0]["gt_masks"].data[0].masks[0][0][0]

98.304

In [2]:
%debug


> /home/aiarhipov/mmdetection/mmdet/models/utils/gaussian_target.py(56)gen_gaussian_target()
     54     top, bottom = min(y, radius), min(height - y, radius + 1)
     55 
---> 56     masked_heatmap = heatmap[y - top:y + bottom, x - left:x + right]
     57     masked_gaussian = gaussian_kernel[radius - top:radius + bottom,
     58                                       radius - left:radius + right]

> /home/aiarhipov/mmdetection/mmdet/models/dense_heads/cycle_centernet_head.py(280)get_targets()
    278                 ctx_int, cty_int = ctx.astype(int) * width_ratio, cty.astype(int) * height_ratio
    279 
--> 280                 gen_gaussian_target(
    281                     heatmap=center_heatmap_target[batch_id, 0],
    282                     center=[ctx_int, cty_int],

102.25
32.0
409.9555555555555
102.48888888888888
102


In [11]:
from mmdet.models.utils import gaussian_radius
radius = gaussian_radius([512/4/8, 512/4/8], min_overlap=0.3)
radius = max(0, int(radius))
radius

3

In [13]:
# Creating a set using string
test_set = set()
test_set.add((1, 2))
test_set.add((1, 3))
# Iterating using for loop
for x, y in test_set:
	print(x, y)


1 2
1 3


In [2]:
%debug

> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/selectors.py(415)select()
    413         ready = []
    414         try:
--> 415             fd_event_list = self._selector.poll(timeout)
    416         except InterruptedError:
    417             return ready

5000
> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/multiprocessing/connection.py(931)wait()
    929 
    930             while True:
--> 931                 ready = selector.select(timeout)
    932                 if ready:
    933                     return [key.fileobj for (key, events) in ready]

4.999996660975739
> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/multiprocessing/connection.py(424)_poll()
    422 
    423     def _poll(self, timeout):
--> 424         r = wait([self], timeout)
    425         return bool(r)
    426 

> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/multiprocessing/connection.py(257)poll()
    255         self._check_closed()
    256         self._check

In [19]:
import torch
hw = torch.zeros((3, 2, 15, 9))
c2v = torch.ones((3, 8, 10, 10)) * 4
v2c = torch.ones((3, 8, 300, 400)) * (-3)


In [20]:
from mmdet.models.utils import gaussian_radius, gen_gaussian_target
from mmdet.models.utils.gaussian_target import get_local_maximum, get_topk_from_heatmap, transpose_and_gather_feat

gen_gaussian_target(
                    heatmap=hw[0, 0],
                    center=[5, 5],
                    radius=2,
                )

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0561, 0.4868, 1.0000, 0.4868, 0.0561, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000,

In [24]:
hw[0, 0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0561, 0.4868, 1.0000, 0.4868, 0.0561, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000,

In [28]:
if hw[0, 0, 5, 5] == 1.:
    print(1)

1


In [15]:
gen_gaussian_target(
                    heatmap=hw[0, 0],
                    center=[5, 5],
                    radius=2,
                )

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0561, 0.4868, 1.0000, 0.4868, 0.0561, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0273, 0.2369, 0.4868, 0.2369, 0.0273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0032, 0.0273, 0.0561, 0.0273, 0.0032, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000,

In [14]:
c2v[0, 0, 0, 0] += 1-torch.exp(np.pi*min(c2v[0, 0, 0, 0], torch.tensor(1.)))

NameError: name 'np' is not defined

In [32]:
c2v[0, 0, ] = 1
c2v[0, 0]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.]])

In [37]:
c2v[0, 0, 0, 0] < 1

tensor(False)

In [26]:
for idx,  a ,b in enumerate(((1, 2), (3, 4))):
    
    print(idx, a ,b)

ValueError: not enough values to unpack (expected 3, got 2)

In [7]:
c2v[0, 0:2, 0, 0]- v2c[0, 0:2, 0, 0]

tensor([7., 7.])

In [13]:
torch.exp(c2v[0, 0, 0, 0])

tensor(54.5981)

In [8]:
torch.norm(c2v[0, 0:2, 0, 0]- v2c[0, 0:2, 0, 0], p=2)

tensor(9.8995)

In [20]:
hw[:, 0, ...] = (- diff[:, 0, ...] + diff[:, 2, ...] + diff[:, 4, ...] - diff[:, 6, ...]) /2
hw[:, 1, ...] = (- diff[:, 1, ...] - diff[:, 3, ...] + diff[:, 5, ...] - diff[:, 7, ...]) /2


In [3]:
t = torch.randn((4, 3))

torch.expit(t)

AttributeError: module 'torch' has no attribute 'expit'

In [1]:
import os.path as osp
from sys import argv

import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/20_cycle_l1/config.py")


set_random_seed(0, deterministic=False)

# Build dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=12.65s)
creating index...
index created!
loading annotations into memory...
Done (t=3.72s)
creating index...
index created!


2023-05-07 20:42:07,275 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=4.67s)
creating index...


2023-05-07 20:42:12,440 - mmdet - INFO - Start running, host: aiarhipov@lorien.atp-fivt.org, work_dir: /home/aiarhipov/centernet/exps/20_cycle_l1_avg_factor_8_2heatmaps_short
2023-05-07 20:42:12,442 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 ------------------

...

2023-05-07 20:42:17,584 - mmdet - WARNING - No meta information found in the runner. 
wandb:   32 of 32 files downloaded.  
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
2023-05-07 20:51:13,915 - mmdet - INFO - Epoch [1][500/1372]	lr: 6.244e-04, eta: 2 days, 11:38:09, time: 1.046, data_time: 0.016, memory: 2448, loss_center_heatmap: 8.7718, loss_offset: 0.1174, loss_c2v: 3.4393, loss_v2c: 0.0227, loss: 12.3512, grad_norm: 72.8073
2023-05-07 21:00:13,240 - mmdet - INFO - Epoch [1][1000/1372]	lr: 1.249e-03, eta: 2 days, 12:25:36, time: 1.079, data_time: 0.010, memory: 2448, loss_center_heatmap: 1.9584, loss_offset: 0.0864, loss_c2v: 1.9862, loss_v2c: 0.0162, loss: 4.0472, grad_norm: 17.3755
2023-05-07 21:07:04,958 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3611/3611, 10.5 task/s, elapsed: 344s, ETA:     0s

2023-05-07 21:18:05,805 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=45.13s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


KeyboardInterrupt: 

In [17]:
x = 2
y = 3
x, y = map(lambda x: 5 if x > 5 else x if x > 0 else 0, (x, y))

In [23]:
(1, 2, 3)* 2

(1, 2, 3, 1, 2, 3)

In [1]:
import os.path as osp
from sys import argv

import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
import subprocess
import random


def get_free_gpu():
    log = str(
        subprocess.check_output("nvidia-smi --format=csv --query-gpu=utilization.gpu,memory.used", shell=True)
    ).split(r"\n")[1:-1]
    free_gpu = []
    for idx, gpu_info in enumerate(log):
        if gpu_info[:-4].split(" %, ")[0] == "0" and gpu_info[:-4].split(" %, ")[1] == "3":
            free_gpu.append(idx)
    if free_gpu:
        return random.choice(free_gpu)
    raise RuntimeError("All gpus are used")

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/25_test_ds/config.py")


set_random_seed(0, deterministic=False)
cfg.gpu_ids = [get_free_gpu()]
val = True
# Build dataset

datasets = build_dataset(cfg.data.train)
# datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]
# # Build the detector
# model = build_detector(cfg.model)
# # Add an attribute for visualization convenience
# model.CLASSES = datasets[0].CLASSES

# # Create work_dir
# mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=55.42s)
creating index...
index created!


In [9]:
datasets[0].keys()

dict_keys(['img_metas', 'img', 'gt_bboxes', 'gt_labels'])

In [10]:
datasets[0]["img_metas"]

DataContainer({'filename': '/home/aiarhipov/datasets/WTW-dataset/train/images/20200211180812076257-0.jpg', 'ori_filename': '20200211180812076257-0.jpg', 'ori_shape': (2132, 1500, 3), 'img_shape': (512, 512, 3), 'pad_shape': (512, 512, 3), 'scale_factor': array([1.1130434, 1.1130434, 1.1130434, 1.1130434], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([103.53 , 116.28 , 123.675], dtype=float32), 'std': array([1., 1., 1.], dtype=float32), 'to_rgb': False}})

In [ ]:
datasets[0]["gt_bboxes"]

In [15]:
datasets[0]["gt_masks"]

DataContainer(PolygonMasks(num_masks=39, height=512, width=360))